In [1]:
import torch
import numpy as np

In [2]:
bikes_numpy = np.loadtxt(r"C:\Users\divya\PyTorchDiaries\data\p1ch4\bike-sharing-dataset\hour-fixed.csv",
                        dtype= np.float32,
                        delimiter = ',',
                        skiprows = 1,
                        converters = {1: lambda x : float(x[8:10])})
                         

In [3]:
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [4]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [5]:
# Dimesnions shape to : day, hour and rest of 17 columns
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [6]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride() # NCL ordering

(torch.Size([730, 17, 24]), (408, 1, 17))

### Manipulating weather variable

In [13]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:,9] # 9th Column was weather conditions

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [16]:
weather_onehot.scatter_(
    dim=1, 
    index=first_day[:,9].unsqueeze(1).long() - 1, # <1>
    value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [18]:
#concat this matrix to original database using cat function
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [19]:
daily_bikes.shape

torch.Size([730, 17, 24])

In [20]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [21]:
daily_weather_onehot.scatter_(
    1,
    daily_bikes[:,9,:].long().unsqueeze(1)-1,
    1.0
)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [23]:
# Concat along the C dimsnsion
daily_bikes = torch.cat((daily_bikes,daily_weather_onehot), dim = 1)

In [24]:
daily_bikes

tensor([[[1.0000e+00, 2.0000e+00, 3.0000e+00,  ..., 2.2000e+01,
          2.3000e+01, 2.4000e+01],
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[2.5000e+01, 2.6000e+01, 2.7000e+01,  ..., 4.5000e+01,
          4.6000e+01, 4.7000e+01],
         [2.0000e+00, 2.0000e+00, 2.0000e+00,  ..., 2.0000e+00,
          2.0000e+00, 2.0000e+00],
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         ...,
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 0.0000e+00,
          0.000

In [25]:
# We could alternatively assume that the ordinal relationship as special values of continuous variable and transform them to run from 0.0 to 1.0
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0)/3.0

In [27]:
# Rescaling variables to [,1] or [-1,1]is sonething often done for all quantative variables
#like temperature in col 10 of dataset

In [28]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min / (temp_max - temp_min)))

In [29]:
# Or subtract mean and devide by stddev
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))/torch.std(temp))

In [30]:
# In the last case, our variable wpuld have 0 mean and 1 std dev
# If it were drawn form Gaussian distribution it would have 68% data between +-1 interavl